In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Activation functions
def sigmoid(x):
    s = 1 / (1 + np.exp(-x))
    return s
    
def relu(x):
    return np.maximum(0, x)

In [24]:
class neural_network:
    def __init__(self, eta, n_hidden, D_input, D_hidden, D_output):
        self.eta = eta
        self.n_iter = n_iter
        self.D_input = D_input        # Input dimension
        self.D_hidden = D_hidden      # Hidden layer dimension
        self.D_output = D_output      # Output dimension

    def parameters(self):
        W1 = np.random.randn(self.D_hidden, self.D_input) * np.sqrt(1 / self.D_input)
        b1 = np.random.randn(self.D_hidden, 1)
        W2 = np.random.randn(self.D_output, self.D_hidden) * np.sqrt(1 / self.D_hidden)
        b2 = np.random.randn(self.D_output, 1)
        return {'W1': W1, 'W2': W2, 'b1': b1, 'b2': b2}

    def ForwardPropagation(self, X, params):
       Z1 = np.dot(params['W1'], X)+params['b1']
       A1 = np.tanh(Z1)
       Z2 = np.dot(params['W2'], A1)+params['b2']
       y = sigmoid(Z2)  
       return y, {'Z1': Z1, 'Z2': Z2, 'A1': A1, 'y': y}
        
    
    def CostEstimate(self, Y, y_hat):
        m = Y.shape[1]  # Number of samples
        cost = -(1 / m) * np.sum(Y * np.log(y_hat) + (1 - Y) * np.log(1 - y_hat))
        return np.squeeze(cost)  # Ensure it's a scalar

    def BackPropagation(self, params, cache, X, Y):
         m = X.shape[1]
    
         # First, retrieve W from the dictionary "parameters".
         W1 = params["W1"]
         W2 = params["W2"]
    
         # Retrieve also A1 and A2 from dictionary "cache".
         A1 = cache["A1"]
         y = cache["y"]
    
    # Backward propagation: calculate partial derivatives denoted as dW1, db1, dW2, db2 for simplicity. 
         dZ2 = y - Y
         dW2 = 1/m * np.dot(dZ2, A1.T)
         db2 = 1/m * np.sum(dZ2, axis = 1, keepdims = True)
         dZ1 = np.dot(W2.T, dZ2) * A1 * (1 - A1)
         dW1 = 1/m * np.dot(dZ1, X.T)
         db1 = 1/m * np.sum(dZ1, axis = 1, keepdims = True)
    
         grads = {"dW1": dW1,
                  "db1": db1,
                  "dW2": dW2,
                  "db2": db2}
    
         return grads

#grads = backward_propagation(parameters, cache, X, Y)

#print("dW1 = " + str(grads["dW1"]))
#print("db1 = " + str(grads["db1"]))
#print("dW2 = " + str(grads["dW2"]))
#print("db2 = " + str(grads["db2"]))

    def UpdateParameters(self, params, grads):
         W1 = parameters["W1"]
         b1 = parameters["b1"]
         W2 = parameters["W2"]
         b2 = parameters["b2"]
        ### END CODE HERE ###
    
       # Retrieve each gradient from the dictionary "grads".
       ### START CODE HERE ### (~ 4 lines of code)
         dW1 = grads["dW1"]
         db1 = grads["db1"]
         dW2 = grads["dW2"]
         db2 = grads["db2"]
       ### END CODE HERE ###
    
       # Update rule for each parameter.
       ### START CODE HERE ### (~ 4 lines of code)
         W1 = W1 - learning_rate * dW1
         b1 = b1 - learning_rate * db1
         W2 = W2 - learning_rate * dW2
         b2 = b2 - learning_rate * db2
       ### END CODE HERE ###
    
         parameters = {"W1": W1,
                       "b1": b1,
                       "W2": W2,
                       "b2": b2}
    
         return parameters
        

In [ ]:
# Define the MLP model
D_input = X_train.shape[1]
print('input size',input_size)
D_hidden = 2
D_output = y_train.shape[1]
# Training parameters
num_epochs = 300 #n_iter
learning_rate = 0.01 #eta
NK = neural_network(learning_rate, num_epoch, D_input, D_hidden, D_output)

 
 # Training loop
for epoch in range(num_epochs):
        params = parameters(X_train)
        # Forward pass
        output = NK.ForwardPropagation(X_train.T, params)

        # Backward pass and parameter update
        gradients = NK.BackPropagation(X_train.T, y_train.T)
        mlp.update_parameters(gradients, learning_rate)

        # Compute and print loss
        loss = np.mean((outputs - y_train.T) ** 2)
        if (epoch + 1) % 300 == 0:
            print(f"Epoch {epoch+1} - Loss: {loss}")